In [1]:
import mysql.connector
import pandas as pd
import glob

class dataPipeline:
    
    def __init__(self):
        self.counter = 0
        self.path = r"InputFiles\*.csv"
    
    def get_db_connection(self):
        
        """
        Creates a database connection to the mysql database
        Parameters
        ----------
        None
        Returns
        -------
        object
            connection object if successful, prints error message otherwise
        """ 
        
        connection = None
    
        try:
            connection = mysql.connector.connect(user='root',
                        password='******',                                             
                         host='127.0.0.1',
                         port='3306',
                         database='ticketsales',auth_plugin='mysql_native_password')
    
        except Exception as error:
            print("Error while connecting to database for job tracker", error)
    
        return connection
    
    def load_third_party(self,connection, file_path_csv):
            
        """
        parse through the csv files in the given path and load the records to the 
        mysql database
        Parameters
        ----------
        connection - db connection object
        file_path_csv - csv folder path
        Returns
        -------
        bool
            True if successful, False otherwise
        """ 
        
        try:
            cursor = connection.cursor()
            
            for fname in glob.glob(file_path_csv):
                df = pd.read_csv(fname, header=None)
                
                for index, row in df.iterrows():
                    ticket_id = str(row[0])
                    trans_date = str(row[1])
                    event_id = str(row[2])
                    event_name = str(row[3])
                    event_date = str(row[4])
                    event_type = str(row[5])
                    event_city = str(row[6])
                    customer_id = str(row[7])
                    price = str(row[8])
                    num_tickets = str(row[9])
                    cursor.execute('INSERT INTO sales(ticket_id,trans_date,event_id,event_name, \
                                    event_date, event_type,event_city,customer_id, \
                                    price,num_tickets) ' \
                                    'VALUES(' + ticket_id  + ',"' + trans_date + '",' + event_id + \
                                    ',"' + event_name + '","' + event_date + '","' + event_type + \
                                    '","' + event_city + '",' + customer_id + \
                                    ',' + price + ',' + num_tickets  + ')') 
                
                self.counter += 1
                
            connection.commit()
            cursor.close()
            
            return True
        
        except Exception as error:
            print("Error while loading csv file to database for job tracker", error)
            
            if cursor:
                cursor.close()          
            
            return False
    
        return connection    
 
    def query_popular_tickets(self,connection):
        
        """
        Query the database and return the top 3 popular tickets event names
        Parameters
        ----------
        connection - db connection object
        Returns
        -------
        list
            True if successful, prints error message otherwise
        """ 
        
        try:        
            # Get the most popular ticket in the past month
            sql_statement = "select event_name,sum(num_tickets) as total_tickets from " \
                            "sales group by event_name order by total_tickets desc limit 3 "
            cursor = connection.cursor()
            cursor.execute(sql_statement)
            records = cursor.fetchall()
            
            result = []
            
            for record in records:
                result.append(record[0])
            
            cursor.close()
            return result
        
        except Exception as error:
            print("Error while querying popular tickets for job tracker", error)

In [2]:
def main():
    
    path = r"InputFiles\*.csv"
    
    # Instantiate the object for the data pipeline
    dp = dataPipeline()
    
    # Get database connection
    conn = dp.get_db_connection()
    
    # Load the csv files to the database
    dp.load_third_party(conn,path)
    
    # Query the top 3 popular records 
    records = dp.query_popular_tickets(conn)
    
    # Print the records
    print(records)
    
if __name__ == "__main__":
    main()

['Washington Spirits vs Sky Blue FC', 'Christmas Spectacular', 'The North American International Auto Show']
